---
title: Skill Gap Analysis
format: html
jupyter: python3
execute:
  echo: true
  warning: false
  error: true
toc: true
code-fold: true
---

In [ ]:
import pandas as pd
df = pd.read_parquet("data/eda.parquet")

## Group 11 Skill 


In [ ]:
import pandas as pd

skills_data = {
    "Name": ["Binderiya", "Pranjul", "Pratham", "Panyang"],
    "Python": [4, 4, 5, 3],
    "SQL": [4, 4, 5, 4],
    "Machine Learning": [2, 3, 2, 2],
    "PySpark": [3, 3, 3, 3],
    "Excel": [4, 5, 5, 4],
    "Data Visualization": [5, 5, 3, 3],
    "Power Bi/ Tableau": [4, 5, 3, 4],
    "Version Control Git": [4, 4, 3, 3],
    "ETL/Data pipeline": [3, 2, 1, 2],
    "Communication": [4, 4, 5, 3],
    "Project Management": [5, 5, 5, 3],
    "Cloud Computing": [4, 4, 2, 2]
}

df_skills = pd.DataFrame(skills_data)
df_skills.set_index("Name", inplace=True)
df_skills

In [ ]:
import plotly.express as px

fig = px.imshow(
    df_skills,
    text_auto=True,                
    color_continuous_scale="YlGnBu",
    aspect="auto"                 
)

fig.update_layout(
    title="Team Skill Levels Heatmap",
    xaxis_title="Skills",
    yaxis_title="Team Members",
    width=700,
    height=400,
    margin=dict(l=50, r=20, t=50, b=50)
)

fig.write_html(
    "figures/skill_gap_plot1.html",
    include_plotlyjs="cdn",
    full_html=False
)

<iframe
  src="figures/skill_gap_plot1.html"
  width="100%"
  height="500"
  style="border:none;"
></iframe>


In [ ]:
import plotly.graph_objects as go
from IPython.display import IFrame
fig = go.Figure()

for name in df_skills.index:
    values = df_skills.loc[name].tolist()
    values += values[:1]  # close the loop
    fig.add_trace(go.Scatterpolar(
        r=values,
        theta=df_skills.columns.tolist() + [df_skills.columns[0]],
        fill='toself',
        name=name
    ))

fig.update_layout(
    polar=dict(radialaxis=dict(visible=True, range=[0, 5])),
    showlegend=True,
    title='Team Skills Radar Chart'
)

fig.write_html(
    "figures/skill_gap_plot2.html",
    include_plotlyjs="cdn",
    full_html=False
)

<iframe
  src="figures/skill_gap_plot2.html"
  width="100%"
  height="500"
  style="border:none;"
></iframe>

### Interactive Radar Chart



From this radar chart visualization we can see that our team has a lot of room for improvement for skills like PySpark and Machine Learning. Also we can see that not a lot of our team mates are confident in their skills in Cloud Computing and ETL. 

## Top Skills


In [ ]:
keywords = ['Data Analyst', 'Business Analyst', 'Data Engineering', 'Deep Learning',
            'Data Science', 'Data Analysis','Data Analytics',  'Market Research Analyst' 
            'LLM', 'Language Model', 'NLP', 'Natural Language Processing',
            'Computer Vision', 'Business Intelligence Analyst', 'Quantitative Analyst', 'Operations Analyst']

match = lambda col: df[col].str.contains('|'.join(keywords), case=False, na=False)

df['DATA_ANALYST_JOB'] = match('TITLE_NAME') \
             | match('SKILLS_NAME') \
             | match('SPECIALIZED_SKILLS_NAME') 
df['DATA_ANALYST_JOB'].value_counts()

In [ ]:
import ast
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

# Safely apply literal_eval only to non-null values
df['SKILLS'] = df['SKILLS_NAME'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [])


data_skills = df[df['DATA_ANALYST_JOB']]['SKILLS'].explode().value_counts().reset_index()
data_skills.columns = ['Skill', 'Count']

fig = px.bar(data_skills, x='Skill', y='Count',
             title="Top Skills",
             labels={'Skill': 'Skill Name', 'Count': 'Frequency'},
             color='Skill')
df_skills.index = df_skills.index.str.strip()

In [ ]:
from collections import defaultdict

# Lowercase everything
team_skills = [s.lower().strip() for s in df_skills.columns]
job_demand_raw = data_skills.copy()
job_demand_raw['Skill'] = job_demand_raw['Skill'].str.lower().str.strip()

# New dict to map cleaned team skill to total count from job postings
skill_demand_map = defaultdict(int)

for _, row in job_demand_raw.iterrows():
    skill_in_posting = row['Skill']
    count = row['Count']
    for team_skill in team_skills:
        if team_skill in skill_in_posting:
            skill_demand_map[team_skill] += count

In [ ]:
team_skills = [s.strip().lower() for s in df_skills.columns]
print("Team skills:", team_skills)
print(job_demand_raw['Skill'].head(10).tolist())
for skill_text in job_demand_raw['Skill'].head(10):
    for team_skill in team_skills:
        if team_skill in skill_text:
            print(f" '{team_skill}' found in: '{skill_text}'")

In [ ]:
for _, row in job_demand_raw.iterrows():
    skill_text = row['Skill']
    count = row['Count']
    for team_skill in team_skills:
        if team_skill in skill_text:  # no regex, just substring
            skill_demand_map[team_skill] += count

job_demand = pd.Series(skill_demand_map)
print(job_demand)

In [ ]:
job_demand = pd.Series(skill_demand_map)
job_demand.name = "Count"
team_avg = df_skills.mean()
team_avg.index = team_avg.index.str.strip().str.lower() 
# Now match only overlapping skills
common_skills = job_demand.index.intersection(team_avg.index)
team_avg = team_avg[common_skills]
job_demand = job_demand[common_skills]

# Normalize job demand
job_demand_normalized = 5 * (job_demand / job_demand.max())
job_demand_normalized.name = "Job Demand (Normalized)"

# Combine
comparison_df = pd.concat([team_avg, job_demand_normalized], axis=1)
comparison_df.columns = ["Team Average Skill", "Job Demand (Normalized)"]
comparison_df["Skill Gap"] = comparison_df["Job Demand (Normalized)"] - comparison_df["Team Average Skill"]
comparison_df.sort_values("Skill Gap", ascending=False, inplace=True)

comparison_df

In [ ]:
comparison_df = comparison_df.reset_index().rename(columns={"index": "Skill"})

In [ ]:
import plotly.express as px

fig = px.bar(
    comparison_df,
    x='Skill',
    y='Skill Gap',
    color='Skill Gap',
    color_continuous_scale='RdBu_r',
    title='Skill Gaps: Job Market Expectations vs. Team Capability',
    labels={'Skill Gap': 'Gap (Job Demand - Team Skill)', 'Skill': 'Skill'},
)

fig.add_hline(y=0, line_dash='dash')
fig.update_layout(
    xaxis_tickangle=-45,
    yaxis_title='Gap (Positive = Market expects more)',
    font=dict(size=13),
    height=500,
    plot_bgcolor='white',
)
fig.write_html(
    "figures/skill_gap_plot3.html",
    include_plotlyjs="cdn",
    full_html=False
)

<iframe
  src="figures/skill_gap_plot3.html"
  width="100%"
  height="500"
  style="border:none;"
></iframe>


This bar chart compares our team’s average proficiency in key data-related skills against job market expectations. Skills with positive values (like communication and SQL) indicate areas where market demand exceeds our current capabilities. On the other hand, negative values highlight areas where the team is ahead or closely aligned with market needs. Notably, skills like Python, cloud computing, and project management show the largest gaps, suggesting priority areas for upskilling.
